In [1]:
import os
os.chdir('../')

In [2]:
import data
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [34]:
cols = ['timestamp','step','action_type','reference']
train = data.train_df('small')[cols]
test = data.test_df('small')[cols]
df = pd.concat([train, test])
del train
del test

In [35]:
df = df.sort_values(['timestamp','step'])

In [36]:
df

,timestamp,step,action_type,reference
30334,1541030432,1,search for destination,"Birmingham, United Kingdom"
79826,1541030471,1,search for destination,"Port Macquarie, Australia"
53289,1541030496,1,search for destination,"Tralee, Ireland"
82743,1541030530,1,clickout item,1054190
78745,1541030539,1,clickout item,44932
712,1541030554,1,search for destination,"Sharjah, United Arab Emirates"
30335,1541030559,2,search for destination,"Solihull, United Kingdom"
29793,1541030578,2,search for destination,"Québec City, Canada"
53290,1541030582,24,interaction item image,39801
30336,1541030583,20,interaction item image,1305491


## Compute the reference popularity incrementally

In [38]:
# find the numeric-reference rows
mask = df.reference.str.isnumeric() == True
references = df[mask].reference.values.astype('int')

In [46]:
# initialize the dictionary with reference - 0
dict_pop = dict( (r,0) for r in references)

In [47]:
# iterate over the rows and set the popularity
df['temporal_popularity'] = 0
for idx,row in tqdm(df[mask].iterrows()):
    ref = int(row.reference)
    if row.action_type == 'clickout item':
        dict_pop[ref] += 1
    else:
        df.at[idx, 'temporal_popularity'] = dict_pop[ref]

In [50]:
df.drop(cols,axis=1)

,temporal_popularity
30334,0
79826,0
53289,0
82743,0
78745,0
712,0
30335,0
29793,0
53290,0
30336,0


## Test the feature

In [3]:
mode = 'small'

from extract_features.reference_temporal_popularity import ReferenceTemporalPopularity
f = ReferenceTemporalPopularity(mode,'cluster_recurrent')

In [4]:
f.join_to(data.train_df(mode))

reference_temporal_popularity feature read


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,temporal_popularity
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0
1,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3,0
2,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0
3,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0
6,00RL8Z82B2Z1,aff3928535f48,1541037542,13,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5,0
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1,0
8,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0
9,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1,0
